# LUMEN DataSci 2021

## Data loading

### Imports

In [1]:
import pandas as pd
import numpy as np
import scipy

### Dataset

In [2]:
dataset = pd.read_csv('LUMEN_DS.csv', sep='|', quotechar='"', encoding='UTF-16LE')
#negative_gm = dataset[dataset['GM%'] < 0]
dataset.shape

/home/msmetko/Projects/lumen/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (16,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(1294962, 33)

## Data cleaning

In [3]:
# List of all the features
for feature_name in dataset.keys():
    print(feature_name)

Manufacturing Region
Manufacturing Location Code
Intercompany
CustomerID
Customer industry
Customer Region
Customer First Invoice Date
Top Customer Group
Item Code
Product family
Product group
Price last modified date in the ERP
Born on date
Make vs Buy
Sales Channel - Internal
Sales Channel - External
Sales Channel - Grouping
Invoice Date
Invoice #
Invoice Line #
Order Date
Order #
Order Line #
Invoiced qty (shipped)
Ordered qty
Invoiced price
Invoiced price (TX)
Cost of part
Material cost of part
Labor cost of part
Overhead cost of part
GM%
# of unique products on a quote


In [4]:
dataset.drop(
    ['CustomerID',
     'Item Code',
     'Invoice #',
     'Order #',
     'Invoiced price (TX)'],
    axis=1, inplace=True)

In [5]:
# Invoiced price, should be > 0
dataset.drop(dataset[dataset['Invoiced price'] <= 0].index, inplace=True)

In [6]:
# Share of NaN's per feature
na_share = dataset.isna().mean()
na_share

Manufacturing Region                   0.022154
Manufacturing Location Code            0.021698
Intercompany                           0.000000
Customer industry                      0.000000
Customer Region                        0.000073
Customer First Invoice Date            0.000000
Top Customer Group                     0.000000
Product family                         0.000000
Product group                          0.098992
Price last modified date in the ERP    0.720977
Born on date                           0.040212
Make vs Buy                            0.013767
Sales Channel - Internal               0.120557
Sales Channel - External               0.120574
Sales Channel - Grouping               0.999983
Invoice Date                           0.000000
Invoice Line #                         0.000000
Order Date                             0.000000
Order Line #                           0.037587
Invoiced qty (shipped)                 0.000000
Ordered qty                            0

`'Price last modified date in the ERP'` has a lot of missing values **(73.8%)**, since a lot of prices never changed. Therefore, we put the oldest change date to all the `NaN`s

In [7]:
column = 'Price last modified date in the ERP'
last_modified = dataset[column]
oldest = last_modified[last_modified.notna()].min()
dataset.loc[last_modified.isna(), column] = oldest
assert not dataset[column].isna().any()

Drop all the features (column) which have more than 9% of `Nan`'s

**TODO: impute them using regressor**

In [8]:
dataset.drop(na_share[na_share > 0.09].index, axis=1, inplace=True)
na_share = dataset.isna().mean()
na_share

Manufacturing Region               0.022154
Manufacturing Location Code        0.021698
Intercompany                       0.000000
Customer industry                  0.000000
Customer Region                    0.000073
Customer First Invoice Date        0.000000
Top Customer Group                 0.000000
Product family                     0.000000
Born on date                       0.040212
Make vs Buy                        0.013767
Invoice Date                       0.000000
Invoice Line #                     0.000000
Order Date                         0.000000
Order Line #                       0.037587
Invoiced qty (shipped)             0.000000
Ordered qty                        0.037587
Invoiced price                     0.007178
Cost of part                       0.017413
GM%                                0.024591
# of unique products on a quote    0.037604
dtype: float64

## Drop all rows with NaN's
**TODO: impute then using regressor**

In [9]:
dataset.dropna(inplace=True)
dataset.shape

(1101992, 20)

In [10]:
#column = 'Manufacturing Location Code'
#vc = dataset[column].value_counts()
#vc

In [11]:
#for region in ['North America', 'Asia', 'Europe']:
#    x = dataset[column].loc[dataset['Manufacturing Region'] == region].unique()
#    s = set(x)
#    print(len(s), s)

In [12]:
#import plotly.express as px
#column = 'Manufacturing Location Code'
#df = dataset[[column, 'GM%']]#.loc[dataset['Manufacturing Region'] == 'Asia']


#fig = px.box(df, x=column, y='GM%')
#fig.show()
#df

In [13]:
from datetime import datetime
import calendar

def transform_order_date(order_date):
    order_date = order_date.apply(datetime.fromisoformat)
    day_of_week = []
    day_of_month = []
    day_of_year = []
    for date in order_date:
        dow = date.weekday()
        date_day = date.day-1
        doy = date.timetuple().tm_yday - 1 # starts from 1
        month_length = calendar.monthrange(date.year, date.month)[1]
        year_length = 365 if not calendar.isleap(date.year) else 366
        day_of_week.append(2*np.pi*dow / 7)
        day_of_month.append(2*np.pi*date_day / month_length)
        day_of_year.append(2*np.pi*doy / year_length)
    dow = np.array(day_of_week)
    dom = np.array(day_of_month)
    doy = np.array(day_of_year)
    return np.cos(dow), np.sin(dow), np.cos(dom), np.sin(dom), np.cos(doy), np.sin(doy)

In [14]:
#date = dataset['Order Date']
#print(date)
#print(date)
#dow_x, dow_y, dom_x, dom_y, doy_x, doy_y = transform_order_date(date)
#dataset['dow_x'] = dow_x
#dataset['dow_y'] = dow_y
#dataset['dom_x'] = dom_x
#dataset['dom_y'] = dom_y
#dataset['doy_x'] = doy_x
#dataset['doy_y'] = doy_y
#dataset.drop(['Order Date'], axis=1, inplace=True)
#dataset.shape

In [15]:
#import plotly.express as px
#column = 'Manufacturing Region'
#df = negative_gm[column]#, 'GM%']]#.loc[dataset['Manufacturing Region'] == 'Asia']


#fig = px.histogram(df, x=column)
#fig.show()


In [16]:
#import plotly.express as px
#column = 'Manufacturing Region'
#df = negative_gm[column]#, 'GM%']]#.loc[dataset['Manufacturing Region'] == 'Asia']


#fig = px.histogram(dataset, x='GM%', log_y=True)
#fig.show()

In [17]:
# List of all the features
for feature_name in dataset.keys():
    print(feature_name, dataset[feature_name].dtype)
df = pd.get_dummies(dataset, 
                    columns=['Manufacturing Region', 'Intercompany', 'Customer industry', 'Product family'])

Manufacturing Region object
Manufacturing Location Code object
Intercompany object
Customer industry object
Customer Region object
Customer First Invoice Date object
Top Customer Group object
Product family object
Born on date object
Make vs Buy object
Invoice Date object
Invoice Line # object
Order Date object
Order Line # float64
Invoiced qty (shipped) float64
Ordered qty float64
Invoiced price float64
Cost of part float64
GM% float64
# of unique products on a quote float64


In [18]:
# remove extra features for clustering
#dataset.drop(
#    ['Manufacturing Location Code',
#     'Intercompany',
#     'Invoice #',
#     'Order #',
#     'Invoiced price (TX)'],
#    axis=1, inplace=True)
cluster_df = dataset.drop(
    ['Manufacturing Location Code',
     'Born on date',
     'Make vs Buy',
     'Customer First Invoice Date',
     'Born on date',
     'Make vs Buy',
     'Invoice Date',
     'Invoice Line #',
     'Order Date',
     'Order Line #',
     'Invoiced qty (shipped)'],
    axis=1, inplace=False)

In [19]:
cluster_df = pd.get_dummies(cluster_df)
cluster_df

,Ordered qty,Invoiced price,Cost of part,GM%,# of unique products on a quote,Manufacturing Region_Asia,Manufacturing Region_Europe,Manufacturing Region_North America,Intercompany_NO,Intercompany_YES,...,Customer industry_IC010,Customer industry_IC011,Customer industry_IC012,Customer Region_Asia,Customer Region_Europe,Customer Region_North America,Top Customer Group_OTHER,Top Customer Group_STAR,Product family_PF001,Product family_PF002
0,499.0,0.3900,0.4173,-0.070000,1.0,0,0,1,0,1,...,0,0,0,1,0,0,1,0,0,1
1,1000.0,0.1400,0.0257,0.816429,3.0,0,0,1,1,0,...,0,0,1,0,0,1,1,0,0,1
3,5000.0,0.2000,0.0000,1.000000,1.0,1,0,0,1,0,...,0,0,0,1,0,0,1,0,0,1
4,5000.0,0.2000,0.0000,1.000000,1.0,1,0,0,1,0,...,0,0,0,1,0,0,1,0,0,1
5,5000.0,0.2000,0.0000,1.000000,1.0,1,0,0,1,0,...,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294956,32.0,36.3200,22.5000,0.380507,5.0,0,0,1,1,0,...,0,0,0,0,0,1,1,0,0,1
1294958,15.0,7.0000,0.7500,0.892857,1.0,0,0,1,1,0,...,0,0,0,0,0,1,1,0,0,1
1294959,10000.0,0.0702,0.0450,0.358974,1.0,0,0,1,1,0,...,0,0,0,1,0,0,1,0,0,1
1294960,3000.0,1.9770,1.1336,0.426606,1.0,0,0,1,1,0,...,0,0,1,0,0,1,1,0,0,1


In [20]:
noto = (cluster_df['GM%'] > 1) | (cluster_df['GM%'] < -1)
cluster_df = cluster_df[~noto]
for feature_name in ['Ordered qty', 'Invoiced price', 'Cost of part']:
    cluster_df.drop(index=cluster_df[(cluster_df[feature_name] <= 0)].index, inplace=True)
    feature = cluster_df[feature_name]
    feature = np.log(feature)
    feature = (feature - feature.mean()) / feature.std()
    cluster_df[feature_name] = feature
cluster_df

/home/msmetko/Projects/lumen/venv/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-20-26c26c8ff8a7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_df[feature_name] = feature
/home/msmetko/Projects/lumen/venv/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

,Ordered qty,Invoiced price,Cost of part,GM%,# of unique products on a quote,Manufacturing Region_Asia,Manufacturing Region_Europe,Manufacturing Region_North America,Intercompany_NO,Intercompany_YES,...,Customer industry_IC010,Customer industry_IC011,Customer industry_IC012,Customer Region_Asia,Customer Region_Europe,Customer Region_North America,Top Customer Group_OTHER,Top Customer Group_STAR,Product family_PF001,Product family_PF002
0,0.026648,-0.390952,-0.193411,-0.070000,1.0,0,0,1,0,1,...,0,0,0,1,0,0,1,0,0,1
1,0.248449,-0.733552,-1.137708,0.816429,3.0,0,0,1,1,0,...,0,0,1,0,0,1,1,0,0,1
11,-0.984192,1.768995,1.718663,0.526436,1.0,1,0,0,1,0,...,0,0,0,1,0,0,1,0,0,1
12,-0.984192,1.765536,1.718663,0.521512,1.0,1,0,0,1,0,...,0,0,0,1,0,0,1,0,0,1
16,0.388283,0.248308,0.253083,0.409060,1.0,1,0,0,1,0,...,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294956,-0.849796,1.125236,1.157479,0.380507,5.0,0,0,1,1,0,...,0,0,0,0,0,1,1,0,0,1
1294958,-1.091550,0.574651,0.005208,0.892857,1.0,0,0,1,1,0,...,0,0,0,0,0,1,1,0,0,1
1294959,0.983135,-0.964391,-0.947931,0.358974,1.0,0,0,1,1,0,...,0,0,0,1,0,0,1,0,0,1
1294960,0.598983,0.151852,0.145153,0.426606,1.0,0,0,1,1,0,...,0,0,1,0,0,1,1,0,0,1


In [21]:
1-len(cluster_df) / len(dataset)

0.15541038410442176

In [22]:
#import plotly.express as px
#column = 'Invoiced price'

#fig = px.histogram(df, x=column, log_y=True)
#fig.show()

In [23]:
from sklearn.cluster import OPTICS

cluster = OPTICS()
cluster.fit(cluster_df)

KeyboardInterrupt: 